In [4]:
# Import libraries.

import json

from IPython.display import Markdown, display

In [8]:
# Define parameters.

PROMPTS = "../data/prompts/zeroshot/taskvine_zeroshot_gpt4o_10.json"

In [9]:
# Analyze a singular prompt.

def analyze(prompt: dict):
    print(prompt["content"])
    display(Markdown(prompt["response"]))

In [10]:
# Load prompts from file.
with open(PROMPTS, "r") as fp:
    prompts = json.load(fp)

# Perform the zeroshot prompting.
for prompt in prompts:
    analyze(prompt)

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date".


```python
import ndcctools.taskvine as vine

def main():
    # Create a Taskvine Manager
    manager = vine.Manager()

    # Create a task to get the current date
    task = vine.Task("date > current_date")

    # Add the task to the manager
    manager.add_task(task)

    # Wait for all tasks to complete
    manager.wait_for_all()

    # Shutdown the manager
    manager.shutdown()

if __name__ == "__main__":
    main()
```

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date".


```python
import ndcctools.taskvine as vine

def main():
    # Create a Taskvine Manager
    manager = vine.Manager()

    # Create a task to get the current date
    task = vine.Task("date > current_date")

    # Add the task to the manager
    manager.add_task(task)

    # Wait for all tasks to complete
    manager.wait_for_all()

    # Shutdown the manager
    manager.shutdown()

if __name__ == "__main__":
    main()
```

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date".


...

```python
import taskvine

def main():
    # Initialize the manager
    manager = taskvine.Manager()

    # Set the password for worker authentication
    with open("taskvine.password", "r") as f:
        password = f.read().strip()
    manager.set_password(password)

    # Enable SSL encryption with given key and certificate
    manager.enable_ssl("key.pem", "certificate.pem")

    # Create a task that gets the current date
    task = taskvine.Task()
    task.set_name("get_current_date")
    task.set_execute("date")
    task.set_output("current_date")

    # Submit the task
    manager.submit(task)

    # Wait for all tasks to complete
    manager.wait_for_all()

    # Finish the manager session
    manager.shutdown()

if __name__ == "__main__":
    main()
```

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date". To prevent an unwanted worker from connecting to the manager, use the password stored in the file "taskvine.password". Further, encrypt communication between the manager and worker with SSL using the "key.pem" and "certificate.pem" files.


```python
import taskvine

def main():
    # Initialize the manager
    manager = taskvine.Manager()

    # Set the password for worker authentication
    with open("taskvine.password", "r") as f:
        password = f.read().strip()
    manager.set_password(password)

    # Enable SSL encryption with given key and certificate
    manager.enable_ssl("key.pem", "certificate.pem")

    # Create a task that gets the current date
    task = taskvine.Task()
    task.set_name("get_current_date")
    task.set_execute("date")
    task.set_output("current_date")

    # Submit the task
    manager.submit(task)

    # Wait for all tasks to complete
    manager.wait_for_all()

    # Finish the manager session
    manager.shutdown()

if __name__ == "__main__":
    main()
```

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date". To prevent an unwanted worker from connecting to the manager, use the password stored in the file "taskvine.password". Further, encrypt communication between the manager and worker with SSL using the "key.pem" and "certificate.pem" files.


```python
import taskvine

def main():
    # Initialize the manager
    manager = taskvine.Manager()

    # Set the password for worker authentication
    with open("taskvine.password", "r") as f:
        password = f.read().strip()
    manager.set_password(password)

    # Enable SSL encryption with given key and certificate
    manager.enable_ssl("key.pem", "certificate.pem")

    # Create a task that gets the current date
    task = taskvine.Task()
    task.set_name("get_current_date")
    task.set_execute("date")
    task.set_output("current_date")

    # Submit the task
    manager.submit(task)

    # Wait for all tasks to complete
    manager.wait_for_all()

    # Finish the manager session
    manager.shutdown()

if __name__ == "__main__":
    main()
```